In [3]:
""" Starter code for logistic regression model to solve OCR task 
with MNIST in TensorFlow
MNIST dataset: yann.lecun.com/exdb/mnist/
Author: Chip Huyen
Prepared for the class CS 20SI: "TensorFlow for Deep Learning Research"
cs20si.stanford.edu
"""
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 10

# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
mnist = input_data.read_data_sets('/home/aschioppa/mnist', one_hot=True) 


Extracting /home/aschioppa/mnist/train-images-idx3-ubyte.gz
Extracting /home/aschioppa/mnist/train-labels-idx1-ubyte.gz
Extracting /home/aschioppa/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/aschioppa/mnist/t10k-labels-idx1-ubyte.gz


In [13]:
# Step 2: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# there are 10 classes for each image, corresponding to digits 0 - 9. 
# Features are of the type float, and labels are of the type int
X = tf.placeholder(tf.float32, shape = [batch_size, 784], name = 'image')
Y = tf.placeholder(tf.float32, shape  = [batch_size, 10], name = 'labels')

In [15]:
?tf.Variable

In [35]:
# Step 3: create weights and bias
# weights and biases are initialized to 0
# shape of w depends on the dimension of X and Y so that Y = X * w + b
# shape of b depends on Y
w = tf.Variable(tf.zeros([784,10]),tf.float32,name='w')
b = tf.Variable(tf.zeros(10),tf.float32,name='b')

In [36]:
# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
# to get the probability distribution of possible label of the image
# DO NOT DO SOFTMAX HERE
logits = tf.matmul(X,w)+b

In [37]:
# Step 5: define loss function
# use cross entropy loss of the real labels with the softmax of logits
# use the method:
# tf.nn.softmax_cross_entropy_with_logits(logits, Y)
# then use tf.reduce_mean to get the mean loss of the batch
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y)
loss = tf.reduce_mean(cross_entropy)

In [40]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

In [43]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())	
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0
        writer = tf.summary.FileWriter('./graphs/linear_reg', sess.graph)
        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
        # TO-DO: run optimizer + fetch loss_batch
            _, loss_batch = sess.run([optimizer,loss],feed_dict = {X : X_batch, Y : Y_batch})

            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!') # should be around 0.35 after 25 epochs

    # test the model
    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(

    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = []

    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y:Y_batch}) 
        total_correct_preds.append(accuracy_batch[0])

    print('Accuracy {0}'.format(sum(total_correct_preds)/mnist.test.num_examples))
    writer.close()

Average loss epoch 0: 1.28818812709
Average loss epoch 1: 0.735672422917
Average loss epoch 2: 0.595747303435
Average loss epoch 3: 0.536725721715
Average loss epoch 4: 0.499518537369
Average loss epoch 5: 0.47104046228
Average loss epoch 6: 0.451088569312
Average loss epoch 7: 0.436180419925
Average loss epoch 8: 0.425406183431
Average loss epoch 9: 0.412334635402
Total time: 11.5582051277 seconds
Optimization Finished!
Accuracy 0.8975
